In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import requi9red module
import sys
  
# append the path of the
# parent directory
sys.path.append("..")

In [3]:
import json
import torch 
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn import CrossEntropyLoss, NLLLoss
import torch.nn.functional as F 
from torch import autograd

import numpy as np 

from transformers.models.luke.configuration_luke import LukeConfig
from transformers import AutoTokenizer, AutoModel, AutoConfig

from entitylinker.lukemodel import LukeModel 
from entitylinker.model_utils import *

from entitylinker.dataset import MedMentionsDataset, Collater
from entitylinker.model import EntityLinker 

In [4]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

if torch.cuda.is_available():
    
    print(torch.cuda.current_device())
    print(torch.cuda.device(0))
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))


0
1
Quadro RTX 5000


In [5]:
input_data_file = "/home/vs428/project/MedMentions/full/data/corpus_pubtator.txt"
train_pmids_file = "/home/vs428/project/MedMentions/full/data/corpus_pubtator_pmids_trng.txt"
test_pmids_file = "/home/vs428/project/MedMentions/full/data/corpus_pubtator_pmids_test.txt"

entity_vocab_file = "/home/vs428/project/MedMentions/full/pretraining5/entity_vocab.jsonl"
entity_embedding_model = "/home/vs428/project/MedMentions/full/pretraining5/model_epoch20.bin"
entity_embedding_metadata = "/home/vs428/project/MedMentions/full/pretraining5/metadata.json"

MAX_LENGTH = 128
MODEL_NAME = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

In [6]:
# Create Tokenizer and DataLoader
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

collater = Collater(tokenizer, max_length = MAX_LENGTH)
train_dataset = MedMentionsDataset(input_data_file, train_pmids_file, tokenizer, entity_vocab_file, 
                             max_length = MAX_LENGTH, stride=0, first_token_ent_only=True)
test_dataset = MedMentionsDataset(input_data_file, test_pmids_file, tokenizer, entity_vocab_file, 
                             max_length = MAX_LENGTH, stride=0, first_token_ent_only=True)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collater)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=collater)
# tokens, bio_tags, entity_ids = next(iter(train_dataloader))


In [7]:

# define optimizer, model, loss etc. 


# with open(entity_embedding_metadata) as f:
#     luke_metadata = json.loads(f.read())

# luke_config = luke_metadata['model_config']
# bert_config = AutoConfig.from_pretrained(MODEL_NAME)

# config = LukeConfig(
#     entity_vocab_size=luke_config['entity_vocab_size'],
#     bert_model_name=luke_config['bert_model_name'],
#     entity_emb_size=luke_config['entity_emb_size'],
#     **bert_config.to_dict(),
# )

# luke_model = LukeModel(config)
# pretrained_entity_embeddings = luke_model.load_state_dict(torch.load(entity_embedding_model, map_location=torch.device('cpu')))
# print(pretrained_entity_embeddings)


In [8]:
# Retrieve pretrained entity embeddings

model_archive = ModelArchive.load(entity_embedding_model)

model_archive.tokenizer
model_archive.entity_vocab
model_archive.bert_model_name
model_archive.config
model_archive.max_mention_length
pretrained_entity_embeddings = model_archive.state_dict['entity_embeddings.entity_embeddings.weight']
pretrained_entity_embeddings = pretrained_entity_embeddings.to(device)

In [9]:
def sim_matrix(a, b, eps=1e-8):
    """
    works on a 3D and 2D array
    """
    a_n, b_n = torch.linalg.norm(a, dim=-1)[:, :, None], torch.linalg.norm(b, dim=-1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.matmul(a_norm, b_norm.transpose(0, 1))        
    return sim_mt        


In [10]:
# define loss function
def mention_entity_loss(mention_pred, entity_pred, bio_tags, entity_ids, 
                        attention_mask, pretrained_entity_embedding, device, lm=0.1):
    # attention_mask torch.Size([27,256])
    # mention_pred torch.Size([27, 256, 3])
    # entity_pred torch.Size([27, 256, 256])
    # bio_tags torch.Size([27, 256])
    # entity_ids torch.Size([27, 256])
    
    # unpadded_men_pred torch.Size([25, 3, 256, 256])
    # unpadded_bio_tags torch.Size([25, 256, 256])    
    
    ### MENTION LOSS

    # first compute all loss
    mention_loss = F.nll_loss(mention_pred.permute(0,2,1).contiguous(), bio_tags, reduction="none")

    # get only the unpadded losses: batch size
    num_unpadded = torch.sum(attention_mask, dim=1)
    masked_mention_loss = torch.where(attention_mask == 1, mention_loss, torch.tensor(0.).float().to(device))
    
    # compute average of the masked loss
    avg_mention_loss = torch.sum(masked_mention_loss, dim=1) / num_unpadded
    avg_mention_loss = torch.sum(avg_mention_loss)/len(avg_mention_loss)

#     print("MENTION LOSS: ", avg_mention_loss)

    
    ### ENTITY LOSS
#     cos_sim = sim_matrix(entity_pred, pretrained_entity_embedding)
#     print("cos_sim", cos_sim.shape)
    # cos_sim torch.Size([27, 256, 34727])
    
    # get all the IDs in entity_ids that aren't -1
    nonzero_ent_ids = entity_ids[entity_ids!=-1]
    # get the true entity embeddings
    # shape: [nonzero_ent_ids, embed_dim]
    true_ent_embeddings = pretrained_entity_embedding[nonzero_ent_ids, :]
    
    # get non-neg1 idxs 
    ent_idxs = (entity_ids != -1).nonzero()
    real_entity_pred = entity_pred[ent_idxs[:, 0], ent_idxs[:, 1], :]    
#     print("true_ent_embeddings", true_ent_embeddings.shape)
#     print(real_entity_pred.shape)
    entity_loss = torch.sum(1 - F.cosine_similarity(true_ent_embeddings, real_entity_pred, dim=1))/true_ent_embeddings.shape[0]
#     print("ENTITY LOSS", entity_loss)
#     max_ents = torch.max(cos_sim, dim=2)
    
    # target_ent_embeddings = pretrained_entity_embedding[entity_ids, :]
    # target_ent_embeddings torch.Size([27, 256, 256])
    # max_ent_idxs = torch.argmax(cos_sim, dim=2)
#     max_ents = self.entity_embedding[max_ent_idxs, :]
    
    
#     entity_pred[ent_idxs]
#     entity_loss = torch.mean((output - target)**2)
    
    

    loss = (lm * avg_mention_loss) + ((1-lm) * entity_loss)
    return loss

In [11]:
# define model, etc. 
model = EntityLinker(model_name='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract',
                     pretrained_entity_embeddings=pretrained_entity_embeddings )

model.to(device)


optimizer = optim.Adam(model.parameters(), lr=0.00002)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#     print(name, param.requires_grad)

In [13]:
def get_bi_spans(batch_tags):
    """Returns a list of N x 2 numpy arrays"""
    batch_tags = batch_tags.cpu().detach().numpy()
    bi_spans = []
    for batch_idx in range(batch_tags.shape[0]):
        b_tags = (batch_tags[batch_idx, :] == 0).astype(int)
        i_tags = (batch_tags[batch_idx, :] == 1).astype(int)
        
        # if there aren't any beginning tags, just return an empty array
        if not b_tags.any():
            bi_spans.append(np.array([]))
            continue
        
        # get all B tag locations
        b_idx, = b_tags.nonzero() 

        # get idxs for I tags
        d = np.diff(i_tags)
        i_idx, = d.nonzero()

        # We need to start things after the change in "condition". Therefore, 
        # we'll shift the index by 1 to the right.
        i_idx += 1

        # add end idxs for all B tags (just interweave +1 of b_idxs)
        b_idx = np.vstack((b_idx,b_idx+1)).reshape((-1,),order='F')    

        if i_tags[0]:
            # If the start of condition is True prepend a 0
            i_idx = np.r_[0, i_idx]

        if i_tags[-1]:
            # If the end of condition is True, append the length of the array
            i_idx = np.r_[i_idx, i_tags.size] # Edit

        # reshape to idxs
        b_idx.shape = (-1,2)
        i_idx.shape = (-1,2)    

        bi_idx = []
        # combine the b and i tags
        for start, stop in b_idx:
            # get idx where I tags start for each B tag, if exists
            i, = np.where(i_idx[:, 0] == stop) 
            if i.size > 0:
                bi_idx.append([start, int(i_idx[i, 1])])
            else:
                bi_idx.append([start, stop])

        bi_spans.append(np.array(bi_idx))

    assert batch_tags.shape[0] == len(bi_spans), f"{batch_tags.shape[0]},{len(bi_spans)},{b_idx}"
    return bi_spans


In [14]:
def compute_conf_matrices(mention_preds, entity_preds, bio_tags, entity_ids, pretrained_entity_embedding):
    '''Computes both macro and micro confusion matrices given torch tensor outputs from our model
    '''
    # mention_pred Size([27, 256, 3])
    # entity_pred torch.Size([27, 256, 256])

    nb_classes = 2
    micro_confusion_matrix = torch.zeros(nb_classes, nb_classes)
    # micro_confusion_matrix [[TP, FN], [FP, TN]]
        
    macro_confusion_matrix = []        
    
    # first we need to find the max of the BIO tagging we have
    pred_bio_tags = torch.argmax(mention_preds, dim=2)
    # pred_bio_tags torch.Size([28, 256])
    
    # create two lists of spans
    pred_span_list = get_bi_spans(pred_bio_tags)
    target_span_list = get_bi_spans(bio_tags)    

    for batch_idx, (pred_spans, target_spans) in enumerate(zip(pred_span_list, target_span_list)):
        tp_matches = 0
        
        batch_confusion_matrix = torch.zeros(nb_classes, nb_classes)

        # do the STRONG MATCHING here

        # first check if there are any predicted spans, if not, we just add a bunch of FNs
        if pred_spans.size <= 0 or pred_spans.shape[0] == 0:
            micro_confusion_matrix[0,1] += target_spans.shape[0]
            batch_confusion_matrix[0,1] += target_spans.shape[0]
            
        # conversely, if the target spans are empty, we add a bunch of FPs instead
        elif target_spans.size <= 0:
            micro_confusion_matrix[1,0] += pred_spans.shape[0]
            batch_confusion_matrix[1,0] += pred_spans.shape[0]            
            
        else:
            for span_idx in range(pred_spans.shape[0]):
                
                if torch.all(torch.tensor(pred_spans[span_idx, :] == target_spans), dim=1).any():
                    seq_idx = pred_spans[span_idx, 0]
                    # we use the start span idx to get the entity embedding and compare it to the pretrained embeds

                    cos_sim = sim_matrix(entity_preds[batch_idx, seq_idx, :].unsqueeze(0).unsqueeze(0), 
                                        pretrained_entity_embedding)
                    # this should return something of size [1, 1, sim_vals]
                    pred_ent_embed_id = torch.argmax(torch.squeeze(cos_sim))
                    target_ent_embed_id = entity_ids[batch_idx, seq_idx]
                    # just to test: 
                    # TODO: REMOVE
    #                 if batch_idx == 1 and seq_idx == 0:
    #                     target_ent_embed_id = 2
                    # TODO: ENDREMOVE
                    if pred_ent_embed_id == target_ent_embed_id:
                        # TP
                        micro_confusion_matrix[0,0] += 1
                        batch_confusion_matrix[0,0] += 1 
                        tp_matches += 1
                    else:
                        # FP
                        micro_confusion_matrix[1,0] += 1
                        batch_confusion_matrix[1,0] += 1
                else:
                    # FP 
                    micro_confusion_matrix[1,0] += 1
                    batch_confusion_matrix[1,0] += 1
        
        # all FNs 
        micro_confusion_matrix[0,1] += target_spans.shape[0] - tp_matches
        batch_confusion_matrix[0,1] += target_spans.shape[0] - tp_matches
        
        # we don't do TNs since we don't need them for P,R,F1
        
        # add on the batch confusion matrix to the batch
        macro_confusion_matrix.append(batch_confusion_matrix)

    return micro_confusion_matrix, macro_confusion_matrix


In [15]:
def compute_metrics(metrics, micro_conf, macro_confs):
    '''Given both the micro and macro confusion matrices, we compute a given set of metrics
    
    Matrix orientation is: 
    |  TP  |  FN |
    |-------------
    |  FP  |  TN |
    '''
    micro_metrics = {}
    macro_metrics = {}
    
    if "precision" in metrics:
        micro_metrics['precision'] = micro_conf[0,0] / (micro_conf[0,0] + micro_conf[1,0])
        macro_metrics['precision'] = torch.mean(torch.tensor([batch_conf[0,0] / (batch_conf[0,0] + batch_conf[1,0]) for batch_conf in macro_confs]))
    if "recall" in metrics:
        micro_metrics['recall'] = micro_conf[0,0] / (micro_conf[0,0] + micro_conf[0,1])
        macro_metrics['recall'] = torch.mean(torch.tensor([batch_conf[0,0] / (batch_conf[0,0] + batch_conf[0,1]) for batch_conf in macro_confs]))
    if "f1" in metrics:
        micro_metrics['f1'] = micro_conf[0,0] / (micro_conf[0,0] + ((1/2) * (micro_conf[0,1] + micro_conf[1,0])))
        macro_metrics['f1'] = torch.mean(torch.tensor([batch_conf[0,0] / (batch_conf[0,0] + ((1/2) * (batch_conf[0,1] + batch_conf[1,0]))) for batch_conf in macro_confs]))
        
    return micro_metrics, macro_metrics


In [16]:
# Train!!
def train(model, train_loader, optimizer, criterion, epoch):

    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        tokenized_text, bio_tags, entity_ids = data
        bio_tags = bio_tags.to(device)
        entity_ids = entity_ids.to(device)
#         print("bio_tags", bio_tags.shape)
#         print("entity_ids", entity_ids.shape)
        # zero the parameter gradients
        optimizer.zero_grad()
#         with autograd.detect_anomaly():

        # forward + backward + optimize
        input_ids = tokenized_text['input_ids'].to(device)
        token_type_ids = tokenized_text['token_type_ids'].to(device)
        attention_mask = tokenized_text['attention_mask'].to(device)
                          
        mention_pred, entity_pred = model(input_ids,token_type_ids,attention_mask)
#         print("ent pred", entity_pred.shape)
        loss = criterion(mention_pred, entity_pred, bio_tags, entity_ids, attention_mask, 
                         pretrained_entity_embeddings, device)
#         print(loss)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 2000 mini-batches
            print('[%d, %5d] train loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0
#         if i == 2:
#             break
    print('Finished Training')

In [17]:
def evaluate(model, test_loader, optimizer, criterion, epoch):
    
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        running_loss = 0.0
        
        for i, data in enumerate(test_loader, 0):
            
            tokenized_text, bio_tags, entity_ids = data
            bio_tags = bio_tags.to(device)
            entity_ids = entity_ids.to(device)
            
    #         with autograd.detect_anomaly():

            # forward + backward + optimize
            input_ids = tokenized_text['input_ids'].to(device)
            token_type_ids = tokenized_text['token_type_ids'].to(device)
            attention_mask = tokenized_text['attention_mask'].to(device)

            mention_pred, entity_pred = model(input_ids,token_type_ids,attention_mask)
    #         print("ent pred", entity_pred.shape)
            loss = criterion(mention_pred, entity_pred, bio_tags, entity_ids, 
                            attention_mask, pretrained_entity_embeddings, device)
    #         print(loss)


            micro_conf_mat, macro_conf_mat = compute_conf_matrices(mention_pred, entity_pred, bio_tags, entity_ids, pretrained_entity_embeddings)
            micro_metrics, macro_metrics = compute_metrics(["precision", "recall","f1"], micro_conf_mat, macro_conf_mat)
            print('-' * 89)
            print(f"Micro-Precision: {micro_metrics['precision']}, Macro-Precision: {macro_metrics['precision']}\n")
            print(f"Micro-Recall: {micro_metrics['recall']}, Macro-Recall: {macro_metrics['recall']}\n")
            print(f"Micro-F1: {micro_metrics['f1']}, Macro-F1: {macro_metrics['f1']}\n")    
            print('-' * 89)


            running_loss += loss.item()
            total_loss += loss.item()
            if i % 5 == 4:    # print every 2000 mini-batches
                print('[%d, %5d] test loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 5))
                running_loss = 0.0
        # whatever i is at is the len of test_loader
        return total_loss / i

In [18]:
num_epochs = 25
import time
import math 

# try:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    epoch_start_time = time.time()
    train(model, train_dataloader, optimizer, mention_entity_loss, epoch)
    val_loss = evaluate(model, test_dataloader, optimizer, mention_entity_loss, epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | valid ppl {:8.2f}'.format(epoch, 
                                      (time.time() - epoch_start_time),
                                       val_loss, math.exp(val_loss)))
    print('-' * 89)        
# except Exception as e:
#     print("An exception occurred", e) 
    

[1,     5] train loss: 1.014
[1,    10] train loss: 1.004
[1,    15] train loss: 1.000
[1,    20] train loss: 0.994
[1,    25] train loss: 0.992
[1,    30] train loss: 0.981
[1,    35] train loss: 0.976
[1,    40] train loss: 0.977
[1,    45] train loss: 0.967
[1,    50] train loss: 0.966
[1,    55] train loss: 0.957
[1,    60] train loss: 0.953
[1,    65] train loss: 0.946
[1,    70] train loss: 0.938
[1,    75] train loss: 0.940
[1,    80] train loss: 0.935
[1,    85] train loss: 0.938
[1,    90] train loss: 0.926
[1,    95] train loss: 0.927
[1,   100] train loss: 0.920
[1,   105] train loss: 0.917
[1,   110] train loss: 0.919
[1,   115] train loss: 0.910
[1,   120] train loss: 0.904
[1,   125] train loss: 0.903
[1,   130] train loss: 0.898
[1,   135] train loss: 0.899
[1,   140] train loss: 0.903
[1,   145] train loss: 0.895
[1,   150] train loss: 0.892
[1,   155] train loss: 0.891
[1,   160] train loss: 0.890
[1,   165] train loss: 0.889
Finished Training
-------------------------

-----------------------------------------------------------------------------------------
Micro-Precision: 0.04222378507256508, Macro-Precision: 0.04913223534822464

Micro-Recall: 0.046475600451231, Macro-Recall: nan

Micro-F1: 0.044247787445783615, Macro-F1: 0.05058671906590462

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.05124653875827789, Macro-Precision: 0.05434131622314453

Micro-Recall: 0.053740013390779495, Macro-Recall: 0.05872342362999916

Micro-F1: 0.05246366560459137, Macro-F1: 0.056032031774520874

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.0560000017285347, Macro-Precision: 0.05208045616745949

Micro-Recall: 0.05944798141717911, Macro-Recall: nan

Micro-F1: 0.05767250

-----------------------------------------------------------------------------------------
Micro-Precision: 0.052945561707019806, Macro-Precision: 0.05247754231095314

Micro-Recall: 0.059117402881383896, Macro-Recall: nan

Micro-F1: 0.05586152523756027, Macro-F1: 0.050713278353214264

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.07126076519489288, Macro-Precision: 0.0769544467329979

Micro-Recall: 0.07698815315961838, Macro-Recall: 0.08303510397672653

Micro-F1: 0.07401382923126221, Macro-F1: 0.07934790849685669

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.05968928709626198, Macro-Precision: 0.06351488828659058

Micro-Recall: 0.06552962213754654, Macro-Recall: nan

Micro-F1: 0.062473

-----------------------------------------------------------------------------------------
Micro-Precision: 0.13096773624420166, Macro-Precision: 0.1308744251728058

Micro-Recall: 0.15048183500766754, Macro-Recall: 0.14916056394577026

Micro-F1: 0.1400482952594757, Macro-F1: 0.1382024735212326

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.1478779911994934, Macro-Precision: 0.13662081956863403

Micro-Recall: 0.173540860414505, Macro-Recall: nan

Micro-F1: 0.1596849262714386, Macro-F1: 0.13950298726558685

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.1679029017686844, Macro-Precision: 0.164861261844635

Micro-Recall: 0.1928737461566925, Macro-Recall: nan

Micro-F1: 0.17952415347099304, 

-----------------------------------------------------------------------------------------
Micro-Precision: 0.14573539793491364, Macro-Precision: 0.13951537013053894

Micro-Recall: 0.16426949203014374, Macro-Recall: 0.1495068371295929

Micro-F1: 0.15444840490818024, Macro-F1: 0.1433221995830536

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.12145748734474182, Macro-Precision: 0.13505952060222626

Micro-Recall: 0.135869562625885, Macro-Recall: nan

Micro-F1: 0.12825994193553925, Macro-F1: 0.13470740616321564

-----------------------------------------------------------------------------------------
[2,    30] test loss: 0.815
-----------------------------------------------------------------------------------------
Micro-Precision: 0.16983695328235626, Macro-Precision: 0.16312408447265625

Micro-Recall: 0.20764119923114777, Macro-Recall: n

...

-----------------------------------------------------------------------------------------
Micro-Precision: 0.3648068606853485, Macro-Precision: 0.3620712459087372

Micro-Recall: 0.37199124693870544, Macro-Recall: nan

Micro-F1: 0.36836403608322144, Macro-F1: 0.3585646450519562

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.29532402753829956, Macro-Precision: nan

Micro-Recall: 0.3412322402000427, Macro-Recall: nan

Micro-F1: 0.31662270426750183, Macro-F1: nan

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.29741379618644714, Macro-Precision: nan

Micro-Recall: 0.34271523356437683, Macro-Recall: nan

Micro-F1: 0.318461537361145, Macro-F1: nan

--------------------------------------------

-----------------------------------------------------------------------------------------
Micro-Precision: 0.40934932231903076, Macro-Precision: 0.41179072856903076

Micro-Recall: 0.4257555902004242, Macro-Recall: 0.42190903425216675

Micro-F1: 0.417391300201416, Macro-F1: 0.4137822985649109

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.35316845774650574, Macro-Precision: 0.34863603115081787

Micro-Recall: 0.3742833733558655, Macro-Recall: 0.3607860207557678

Micro-F1: 0.36341947317123413, Macro-F1: 0.3520265221595764

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.41239479184150696, Macro-Precision: 0.40856191515922546

Micro-Recall: 0.4139784872531891, Macro-Recall: 0.412648826837539

-----------------------------------------------------------------------------------------
Micro-Precision: 0.2772277295589447, Macro-Precision: nan

Micro-Recall: 0.341143399477005, Macro-Recall: nan

Micro-F1: 0.30588236451148987, Macro-F1: 0.2754981219768524

-----------------------------------------------------------------------------------------
[19,    55] test loss: 0.633
-----------------------------------------------------------------------------------------
| end of epoch  18 | time: 115.55s | valid loss  0.64 | valid ppl     1.89
-----------------------------------------------------------------------------------------
[20,     5] train loss: 0.603
[20,    10] train loss: 0.588
[20,    15] train loss: 0.589
[20,    20] train loss: 0.593
[20,    25] train loss: 0.593
[20,    30] train loss: 0.583
[20,    35] train loss: 0.594
[20,    40] train loss: 0.598
[20,    45] train loss: 0.602
[20,    50] train loss: 0.599
[20,    55] train loss: 0.599
[20,    60] train loss: 0.584
[20,

-----------------------------------------------------------------------------------------
Micro-Precision: 0.3705882430076599, Macro-Precision: nan

Micro-Recall: 0.3649529218673706, Macro-Recall: 0.34689605236053467

Micro-F1: 0.36774900555610657, Macro-F1: 0.3475441038608551

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.33598726987838745, Macro-Precision: 0.3238956928253174

Micro-Recall: 0.3425324559211731, Macro-Recall: 0.32822805643081665

Micro-F1: 0.3392283022403717, Macro-F1: 0.32192865014076233

-----------------------------------------------------------------------------------------
[20,    20] test loss: 0.636
-----------------------------------------------------------------------------------------
Micro-Precision: 0.3614063858985901, Macro-Precision: nan

Micro-Recall: 0.39855724573135376, Macro-Recall: nan

Micro-F1: 0.37

-----------------------------------------------------------------------------------------
Micro-Precision: 0.31506848335266113, Macro-Precision: 0.3247487246990204

Micro-Recall: 0.30363035202026367, Macro-Recall: 0.30446839332580566

Micro-F1: 0.30924370884895325, Macro-F1: 0.3106740415096283

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.34696969389915466, Macro-Precision: 0.3525993824005127

Micro-Recall: 0.364359587430954, Macro-Recall: nan

Micro-F1: 0.3554520905017853, Macro-F1: 0.346609890460968

-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
Micro-Precision: 0.4028925597667694, Macro-Precision: 0.3859015703201294

Micro-Recall: 0.3622291088104248, Macro-Recall: 0.3434162139892578

Micro-F1: 0.3814

In [19]:
x = torch.randn(2,3,5)
m = torch.nn.Softmax(dim=2)
print(x.shape)
m(x)
# torch.nn.Softmax(x, dim=0)

torch.Size([2, 3, 5])


tensor([[[0.0733, 0.0961, 0.2756, 0.3893, 0.1657],
         [0.3559, 0.0187, 0.1055, 0.1910, 0.3289],
         [0.1496, 0.7128, 0.0404, 0.0556, 0.0417]],

        [[0.1742, 0.0728, 0.0993, 0.1983, 0.4553],
         [0.0665, 0.0258, 0.0714, 0.8028, 0.0336],
         [0.1200, 0.0869, 0.1828, 0.1306, 0.4798]]])

In [20]:
import torch.nn as nn
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos(input1, input2).shape

torch.Size([100])

In [21]:
# in1 shape [batch_size, seq_len, embed_size]
# in1 shape [16, 256, 256]
# in2 shape [1, entity_vocab, embed_size]
# in2 shape [37456, 256]
import torch
from scipy import spatial
import numpy as np

In [22]:
a = torch.randn(16, 256, 256)
# a = torch.randn(3, 5)
b = torch.randn(37456, 256) # different row number, for the fun

In [23]:
%%timeit
# Given that cos_sim(u, v) = dot(u, v) / (norm(u) * norm(v))
#                          = dot(u / norm(u), v / norm(v))
# We fist normalize the rows, before computing their dot products via transposition:
a_norm = a / a.norm(dim=2)[:, :, None]
# a_norm = a / a.norm(dim=1)[:, None]
b_norm = b / b.norm(dim=1)[:, None]
res = torch.matmul(a_norm, b_norm.transpose(0,1))
# print(res.shape)
print(res)
#  0.9978 -0.9986 -0.9985
# -0.8629  0.9172  0.9172

tensor([[[ 7.0350e-02, -7.8589e-02, -5.9236e-03,  ..., -1.6681e-01,
           4.4659e-02, -6.5032e-02],
         [ 4.3987e-02,  1.0746e-02,  4.8974e-02,  ..., -4.6926e-02,
          -1.3284e-02,  1.0489e-02],
         [-5.6106e-02,  7.1678e-02, -9.5512e-02,  ..., -5.5202e-02,
           3.6239e-02, -1.0801e-02],
         ...,
         [-7.2728e-02, -7.9300e-02, -2.6555e-02,  ..., -1.0437e-01,
           1.0003e-01,  1.4226e-01],
         [ 1.3397e-01, -2.2071e-02,  1.0243e-02,  ..., -3.9050e-02,
          -5.8970e-02,  7.0246e-02],
         [-9.6256e-02, -6.0374e-03, -1.7477e-01,  ...,  7.8413e-02,
          -3.0595e-02,  4.8560e-02]],

        [[ 5.6643e-02, -2.9540e-02, -3.3895e-04,  ...,  1.0345e-02,
           7.6199e-02,  2.6074e-02],
         [ 3.5414e-02, -3.5726e-02, -5.6459e-02,  ..., -8.8922e-02,
           2.9471e-02,  1.2311e-02],
         [ 6.3935e-02,  1.7522e-02, -2.8349e-02,  ..., -5.7184e-02,
           4.7690e-02, -2.8668e-02],
         ...,
         [ 7.1103e-02,  1

tensor([[[ 7.0350e-02, -7.8589e-02, -5.9236e-03,  ..., -1.6681e-01,
           4.4659e-02, -6.5032e-02],
         [ 4.3987e-02,  1.0746e-02,  4.8974e-02,  ..., -4.6926e-02,
          -1.3284e-02,  1.0489e-02],
         [-5.6106e-02,  7.1678e-02, -9.5512e-02,  ..., -5.5202e-02,
           3.6239e-02, -1.0801e-02],
         ...,
         [-7.2728e-02, -7.9300e-02, -2.6555e-02,  ..., -1.0437e-01,
           1.0003e-01,  1.4226e-01],
         [ 1.3397e-01, -2.2071e-02,  1.0243e-02,  ..., -3.9050e-02,
          -5.8970e-02,  7.0246e-02],
         [-9.6256e-02, -6.0374e-03, -1.7477e-01,  ...,  7.8413e-02,
          -3.0595e-02,  4.8560e-02]],

        [[ 5.6643e-02, -2.9540e-02, -3.3895e-04,  ...,  1.0345e-02,
           7.6199e-02,  2.6074e-02],
         [ 3.5414e-02, -3.5726e-02, -5.6459e-02,  ..., -8.8922e-02,
           2.9471e-02,  1.2311e-02],
         [ 6.3935e-02,  1.7522e-02, -2.8349e-02,  ..., -5.7184e-02,
           4.7690e-02, -2.8668e-02],
         ...,
         [ 7.1103e-02,  1

tensor([[[ 7.0350e-02, -7.8589e-02, -5.9236e-03,  ..., -1.6681e-01,
           4.4659e-02, -6.5032e-02],
         [ 4.3987e-02,  1.0746e-02,  4.8974e-02,  ..., -4.6926e-02,
          -1.3284e-02,  1.0489e-02],
         [-5.6106e-02,  7.1678e-02, -9.5512e-02,  ..., -5.5202e-02,
           3.6239e-02, -1.0801e-02],
         ...,
         [-7.2728e-02, -7.9300e-02, -2.6555e-02,  ..., -1.0437e-01,
           1.0003e-01,  1.4226e-01],
         [ 1.3397e-01, -2.2071e-02,  1.0243e-02,  ..., -3.9050e-02,
          -5.8970e-02,  7.0246e-02],
         [-9.6256e-02, -6.0374e-03, -1.7477e-01,  ...,  7.8413e-02,
          -3.0595e-02,  4.8560e-02]],

        [[ 5.6643e-02, -2.9540e-02, -3.3895e-04,  ...,  1.0345e-02,
           7.6199e-02,  2.6074e-02],
         [ 3.5414e-02, -3.5726e-02, -5.6459e-02,  ..., -8.8922e-02,
           2.9471e-02,  1.2311e-02],
         [ 6.3935e-02,  1.7522e-02, -2.8349e-02,  ..., -5.7184e-02,
           4.7690e-02, -2.8668e-02],
         ...,
         [ 7.1103e-02,  1

In [24]:
# %%timeit
# # -------
# # Let's verify with numpy/scipy if our computations are correct:
# a_n = a.numpy()
# b_n = b.numpy()
# res_n = np.zeros((3,5,50))

# for i in range(3):
#     for j in range(5):
#         for k in range(50):
#             # cos_sim(u, v) = 1 - cos_dist(u, v)
#             res_n[i, j, k] = 1 - spatial.distance.cosine(a_n[i, j], b_n[k])
# print(res_n)
# # [[ 0.9978022  -0.99855876 -0.99854881]
# #  [-0.86285472  0.91716063  0.9172349 ]]


from sklearn.metrics.pairwise import cosine_similarity
a_n = a.numpy()
b_n = b.numpy()

res_n = np.zeros((16,256, 37456))
for i in range(3):
    res_n[i, :, :] = cosine_similarity(a_n[i,:,:], b_n)
res_n

array([[[ 7.03498796e-02, -7.85887539e-02, -5.92361670e-03, ...,
         -1.66807175e-01,  4.46585976e-02, -6.50324896e-02],
        [ 4.39871177e-02,  1.07464558e-02,  4.89737988e-02, ...,
         -4.69259731e-02, -1.32835405e-02,  1.04894387e-02],
        [-5.61063439e-02,  7.16783926e-02, -9.55120176e-02, ...,
         -5.52019738e-02,  3.62394527e-02, -1.08005898e-02],
        ...,
        [-7.27276579e-02, -7.93004110e-02, -2.65550315e-02, ...,
         -1.04372263e-01,  1.00031167e-01,  1.42261788e-01],
        [ 1.33971170e-01, -2.20705699e-02,  1.02428058e-02, ...,
         -3.90496552e-02, -5.89703135e-02,  7.02463835e-02],
        [-9.62561071e-02, -6.03739591e-03, -1.74768373e-01, ...,
          7.84125999e-02, -3.05945352e-02,  4.85599861e-02]],

       [[ 5.66428304e-02, -2.95396857e-02, -3.38946324e-04, ...,
          1.03453314e-02,  7.61987343e-02,  2.60740984e-02],
        [ 3.54144052e-02, -3.57257277e-02, -5.64586632e-02, ...,
         -8.89221430e-02,  2.94706896e

In [25]:
# a_n.shape
b_n.shape

(37456, 256)

In [26]:
matrix1 = np.array([[[1,1,1],[1,1,1],[1,2,1]],[[1,2,1],[1,1,1],[1,1,1]]])
matrix2 = np.array([[1,1,1],[1,2,1], [1,2,1], [1,2,1]])
print(matrix1.shape)
print(matrix2.shape)

(2, 3, 3)
(4, 3)


In [27]:
import scipy.spatial as sp
out = np.zeros((2,3,4))
for i in range(2):
    out[i, :, :] = cosine_similarity(matrix1[i], matrix2, 'cosine')

In [28]:
out.shape

(2, 3, 4)

In [29]:
out

array([[[1.        , 0.94280904, 0.94280904, 0.94280904],
        [1.        , 0.94280904, 0.94280904, 0.94280904],
        [0.94280904, 1.        , 1.        , 1.        ]],

       [[0.94280904, 1.        , 1.        , 1.        ],
        [1.        , 0.94280904, 0.94280904, 0.94280904],
        [1.        , 0.94280904, 0.94280904, 0.94280904]]])

In [30]:
import numpy as np
z = torch.randint(-1,3, size=(10,)) 
print(z)


tensor([ 1,  2, -1,  0,  0,  1,  0,  1,  2,  0])


In [31]:
x = torch.tensor([[1,4,6],[3,5,6]])
embedding = torch.randint(10, size=(10, 10))
embedding

tensor([[8, 3, 2, 5, 1, 9, 6, 8, 4, 9],
        [2, 6, 8, 1, 5, 1, 6, 5, 7, 8],
        [4, 3, 3, 5, 7, 0, 7, 1, 2, 3],
        [3, 7, 5, 1, 8, 3, 5, 7, 9, 8],
        [6, 8, 4, 1, 0, 3, 5, 6, 6, 3],
        [2, 4, 8, 4, 2, 3, 7, 6, 3, 6],
        [8, 3, 3, 8, 2, 9, 2, 5, 7, 7],
        [0, 8, 4, 3, 9, 5, 3, 9, 0, 6],
        [6, 8, 6, 0, 2, 0, 7, 5, 4, 8],
        [9, 6, 7, 7, 1, 8, 6, 4, 2, 9]])

In [32]:
embedding[x, :]

tensor([[[2, 6, 8, 1, 5, 1, 6, 5, 7, 8],
         [6, 8, 4, 1, 0, 3, 5, 6, 6, 3],
         [8, 3, 3, 8, 2, 9, 2, 5, 7, 7]],

        [[3, 7, 5, 1, 8, 3, 5, 7, 9, 8],
         [2, 4, 8, 4, 2, 3, 7, 6, 3, 6],
         [8, 3, 3, 8, 2, 9, 2, 5, 7, 7]]])

In [33]:
x = torch.randint(-1, 5, size  = (10,10))
idxs = (x == -1).nonzero()
print(x)
idxs

# new_arr_0 = x[x!=-1]
# print(idxs)
# new_arr_0

# ##### 
# x = torch.randint(-1, 5, size  = (5,10)).float()
# y = torch.randint(-1, 5, size  = (5,10)).float()
# import torch.nn.functional as F
# F.cosine_similarity(x, y, dim=1)

tensor([[ 3, -1,  3,  4,  4,  0,  3, -1,  0,  0],
        [ 2,  0,  0,  1,  1,  3, -1,  4,  4, -1],
        [ 2,  2,  4,  2,  2, -1, -1,  0,  2,  1],
        [ 1,  4,  1,  3,  2,  2, -1,  4,  1,  2],
        [-1,  2,  4,  2,  1,  3, -1,  1,  1,  3],
        [-1,  2,  2,  4,  1,  0,  3,  3,  1,  0],
        [ 1,  4,  4, -1, -1,  2,  2,  3, -1,  2],
        [ 2,  2,  4, -1, -1,  1,  0,  4,  4,  4],
        [-1,  0,  4,  3,  2,  2,  2,  0, -1,  4],
        [ 2,  2,  1,  3, -1,  2,  4,  1,  4,  4]])


tensor([[0, 1],
        [0, 7],
        [1, 6],
        [1, 9],
        [2, 5],
        [2, 6],
        [3, 6],
        [4, 0],
        [4, 6],
        [5, 0],
        [6, 3],
        [6, 4],
        [6, 8],
        [7, 3],
        [7, 4],
        [8, 0],
        [8, 8],
        [9, 4]])

In [34]:
y = torch.randint(10, size=(10, 10, 10))
y[idxs[:, 0], idxs[:, 1]].shape
y[idxs[:, 0], idxs[:, 1]]

tensor([[5, 6, 8, 6, 8, 5, 1, 1, 1, 4],
        [6, 1, 1, 7, 8, 9, 9, 7, 5, 1],
        [3, 0, 6, 8, 3, 9, 3, 2, 2, 4],
        [0, 9, 8, 1, 4, 3, 1, 0, 2, 0],
        [7, 5, 1, 4, 3, 6, 3, 2, 4, 0],
        [0, 1, 2, 8, 1, 2, 3, 1, 8, 5],
        [6, 6, 4, 6, 8, 1, 3, 1, 3, 8],
        [1, 7, 4, 2, 7, 0, 6, 1, 7, 8],
        [4, 7, 2, 5, 0, 2, 2, 1, 5, 2],
        [7, 5, 9, 3, 4, 5, 7, 1, 2, 5],
        [0, 9, 4, 1, 6, 0, 9, 9, 5, 6],
        [7, 4, 2, 8, 8, 1, 8, 3, 6, 0],
        [5, 0, 3, 4, 8, 1, 9, 1, 7, 5],
        [3, 9, 2, 4, 7, 1, 2, 9, 8, 6],
        [2, 7, 1, 9, 2, 3, 1, 9, 9, 3],
        [3, 2, 9, 3, 6, 2, 3, 3, 6, 6],
        [8, 7, 8, 9, 7, 5, 0, 3, 7, 4],
        [4, 2, 4, 4, 3, 4, 7, 4, 6, 7]])

In [35]:
import torch.nn as nn
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()
# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)
# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
output = loss(m(conv(data)), target)
output.backward()

In [36]:
print(target.shape)
m(conv(data)).shape

torch.Size([5, 8, 8])


torch.Size([5, 4, 8, 8])

In [37]:
# attention_mask torch.Size([27,256])
# mention_pred torch.Size([27, 256, 3])
# entity_pred torch.Size([27, 256, 256])
# bio_tags torch.Size([27, 256])
# entity_ids torch.Size([27, 256])

# unpadded_men_pred torch.Size([25, 3, 256, 256])
# unpadded_bio_tags torch.Size([25, 256, 256])    


attention_mask = torch.randint(0,2,size=(3,5))
# mention_pred = torch.randn(3,5)
losses = torch.randn(3,5)
# bio_tags = torch.randn(3,5)
print(attention_mask)
print(losses)
# print(bio_tags)
num_unpadded = torch.sum(attention_mask, dim=1)
print("num_unpadded",num_unpadded)
x = torch.where(attention_mask == 1, losses, torch.tensor(0.).float())
print(torch.sum(x, dim=1))
torch.sum(torch.sum(x, dim=1)/num_unpadded)/len(x)
# print(torch.sum(x, dim=1))
# torch.sum(torch.sum(x, dim=1) / num_unpadded)

#     unpadded_men_pred = mention_pred[attention_mask,:].permute(0,3,1,2).contiguous()
# mention_pred[attention_mask,:]

    
    
    

tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1]])
tensor([[ 1.6735,  0.2509,  0.8096,  0.5903,  0.3839],
        [ 0.0505,  1.0096,  0.4159, -0.8981, -2.6978],
        [ 1.3066, -0.6792,  0.1163,  0.2207, -0.2862]])
num_unpadded tensor([0, 4, 5])
tensor([ 0.0000, -1.2217,  0.6781])


tensor(nan)

In [38]:
import numpy as np
# Construct a random 50x50 RGB image    
image = np.random.random((50, 50, 3))

# Construct mask according to some condition;
# in this case, select all pixels with a red value > 0.3
mask = image[..., 0] > 0.3
mask.shape

(50, 50)

In [39]:
def compute_metrics(mention_preds, entity_preds, bio_tags, entity_ids, pretrained_entity_embedding):
    '''Computes both macro and micro confusion matrices given torch tensor outputs from our model
    '''
    # mention_pred Size([27, 256, 3])
    # entity_pred torch.Size([27, 256, 256])

    nb_classes = 2
    micro_confusion_matrix = torch.zeros(nb_classes, nb_classes)
    # micro_confusion_matrix [[TP, FN], [FP, TN]]
#     for t, p in zip(classes.view(-1), preds.view(-1)):
#         confusion_matrix[t.long(), p.long()] += 1
        
    macro_confusion_matrix = []        
    
    # first we need to find the max of the BIO tagging we have
    pred_bio_tags = torch.argmax(mention_preds, dim=2)
    # pred_bio_tags torch.Size([28, 256])
    
    # TODO: Remove after testing
    # pred_bio_tags = torch.randint(0,2, size=(28,256))

    # create two lists of spans
    pred_span_list = get_bi_spans(pred_bio_tags)
    target_span_list = get_bi_spans(bio_tags)    
    
    for batch_idx, (pred_spans, target_spans) in enumerate(zip(pred_span_list, target_span_list)):
        tp_matches = np.zeros(target_spans.shape[0], dtype=bool)
        
        batch_confusion_matrix = torch.zeros(nb_classes, nb_classes)

        # do the STRONG MATCHING here
        for span_idx in range(pred_spans.shape[0]):
            if torch.all(torch.tensor(pred_spans[span_idx, :] == target_spans), dim=1).any():
                seq_idx = pred_spans[span_idx, 0]
                # we use the start span idx to get the entity embedding and compare it to the pretrained embeds
#                 print(entity_preds[batch_idx, seq_idx, :].shape)
                cos_sim = sim_matrix(entity_preds[batch_idx, seq_idx, :].unsqueeze(0).unsqueeze(0), 
                                     pretrained_entity_embedding)
                # this should return something of size [1, 1, sim_vals]
                pred_ent_embed_id = torch.argmax(torch.squeeze(cos_sim))
                target_ent_embed_id = entity_ids[batch_idx, seq_idx]
                # just to test: 
                # TODO: REMOVE
                if batch_idx == 1 and seq_idx == 0:
                    target_ent_embed_id = 2
                # TODO: ENDREMOVE
                if pred_ent_embed_id == target_ent_embed_id:
                    # TP
                    micro_confusion_matrix[0,0] += 1
                    batch_confusion_matrix[0,0] += 1 
                    tp_matches[span_idx] = True
                else:
                    # FP
                    micro_confusion_matrix[1,0] += 1
                    batch_confusion_matrix[1,0] += 1
            else:
                # FP 
                micro_confusion_matrix[1,0] += 1
                batch_confusion_matrix[1,0] += 1

        print(tp_matches)        
        micro_confusion_matrix[0,1] += np.sum(~tp_matches)
        batch_confusion_matrix[0,1] += np.sum(~tp_matches)

        macro_confusion_matrix.append(batch_confusion_matrix)

        print(micro_confusion_matrix)


In [40]:
batch_size = 2
ent_vocab_size = 8
import torch.nn.functional as F

metric_list = ""
mention_preds = torch.tensor(np.random.rand(batch_size, 5, 3))
# mention_preds = F.softmax(torch.tensor(mention_preds), dim=2)
print(mention_preds)
entity_preds = torch.tensor(np.random.rand(batch_size, 5, 5))
bio_tags = torch.tensor(np.random.randint(0,3, size=(batch_size, 5)))
print(bio_tags.shape)
print(bio_tags)
entity_ids = torch.tensor(np.random.randint(ent_vocab_size, size=(batch_size, 5)))
print(entity_ids)
pretrained_entity_embedding = torch.tensor(np.random.rand(ent_vocab_size, 5))
print(pretrained_entity_embedding.shape)
# mention_pred Size([27, 256, 3])
# entity_pred torch.Size([27, 256, 256])
# bio_tags torch.Size([27, 256])
# entity_ids torch.Size([27, 256])

tensor([[[0.0280, 0.4551, 0.4349],
         [0.7973, 0.9785, 0.9968],
         [0.7352, 0.4448, 0.7065],
         [0.1659, 0.4560, 0.9020],
         [0.4978, 0.8070, 0.2581]],

        [[0.2938, 0.4054, 0.2894],
         [0.7827, 0.5111, 0.3319],
         [0.8681, 0.3724, 0.9933],
         [0.0949, 0.5764, 0.4387],
         [0.0866, 0.0764, 0.4486]]], dtype=torch.float64)
torch.Size([2, 5])
tensor([[1, 0, 2, 1, 1],
        [2, 0, 0, 1, 0]])
tensor([[5, 1, 2, 2, 4],
        [6, 1, 5, 7, 0]])
torch.Size([8, 5])


In [41]:
# bio_tags[1,0] = 2
# bio_tags[1,4] = 2
entity_ids[0,2] = -1
entity_ids[1,1] = -1
entity_ids[1, 3] = -1
entity_ids[1,4] = -1
print(bio_tags)
entity_ids

tensor([[1, 0, 2, 1, 1],
        [2, 0, 0, 1, 0]])


tensor([[ 5,  1, -1,  2,  4],
        [ 6, -1,  5, -1, -1]])

In [42]:
compute_metrics(metric_list, mention_preds, entity_preds, bio_tags, entity_ids, pretrained_entity_embedding)

TypeError: compute_metrics() takes 5 positional arguments but 6 were given

In [ ]:
x = torch.tensor([[0, 0, 1, 0, 0],[0, 2, 0, 2, 2]]) ; print(x)
y = torch.tensor([[1, 0, 2, 2, 1], [0, 2, 0, 1, 2]]) ; y

In [ ]:
x_span = get_bi_spans(x)
y_span = get_bi_spans(y)
print(x_span)
print(y_span)

In [ ]:
print(y_span[0][0,:].shape)
print(x_span[0].shape)
# (y_span[0][0,:] ==  x_span[0]).shape
torch.all(torch.tensor(y_span[0][0,:] == x_span[0]), dim=1).any()

In [ ]:
if(np.array([]).size <= 0):
    print("X")

In [ ]:
x = torch.tensor([-0.3616, -0.6805])
torch.sum(x)/2